In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [6]:
url_base = "https://www.gasolinamx.com/estado/ciudad-de-mexico-cdmx/"
tipo_combustible = ["Magna", "Premium", "Diesel"]
delegaciones_cdmx = ['alvaro-obregon', 'azcapotzalco', 'benito-juarez', 'coyoacan', 'cuajimalpa-de-morelos', 'cuauhtemoc',
                     'gustavo-a-madero', 'iztacalco', 'iztapalapa', 'la-magdalena-contreras', 'miguel-hidalgo', 'milpa-alta', 
                     'tlahuac', 'tlalpan', 'venustiano-carranza', 'xochimilco']
fecha = datetime.datetime.now().strftime("%d-%m-%Y")
data = []

In [7]:
def obtener_contneido_pagina(url):
    pagina = requests.get(url)
    return pagina.content

In [8]:
def genera_soup(html):
    return BeautifulSoup(html, "html.parser") 

In [9]:
def procesar_pagina(soup, fecha, delegacion):

    tr_magna = soup.find("tr", class_ = "bg-green")
    precio_magna = tr_magna.text[6:].strip()

    tr_premium = soup.find("tr", class_ = "bg-red")
    precio_premium = tr_premium.text[9:].strip()

    tr_diesel = soup.find("tr", class_ = "bg-black")
    precio_diesel = tr_diesel.text[7:].strip()

    precios = [precio_magna, precio_premium, precio_diesel]
    
    for i in range(len(precios)):
        data.append({"Fecha" : fecha,
                     "Delegacion" : delegacion,
                     "Combustible" : tipo_combustible[i],
                     "Precio" : precios[i]})
      
    return data

In [10]:
def manejar_paginacion(url_base):
    for delegacion in delegaciones_cdmx:
        url = url_base + delegacion
        html = obtener_contneido_pagina(url)
        soup = genera_soup(html)
        procesar_pagina(soup, fecha, delegacion)
    return pd.DataFrame(data)

In [42]:
df_hoy = manejar_paginacion(url_base)
def actualiza_base(df_hoy):
    df_base = pd.read_csv("../csv_gasolina/Base.csv")
    df_nuevo = pd.concat([df_base, df_hoy])
    df_nuevo.to_csv("../csv_gasolina/" + fecha + " actualizacion.csv")

In [43]:
df_nuevo = actualiza_base(df_hoy)

In [ ]:
# Esta bien pero ahora hay que actualizar el proyecto a la mezcla mexicana
# https://www.banxico.org.mx/apps/gc/precios-spot-del-petroleo-gra.html

https://j2logo.com/python/listar-directorio-en-python/#:~:text=Para%20listar%20o%20recorrer%20un,archivos%20y%20carpetas%20que%20contiene.


Para listar o recorrer un directorio en Python basta con usar la función listdir() del módulo os. Esta función recibe como argumento una ruta del sistema de ficheros y devuelve una lista con los nombres de los archivos y carpetas que contiene. Si no se pasa ningún argumento, el directorio de referencia es la carpeta actual.

In [11]:
import os
documentos = os.listdir("../csv_gasolina")
documentos

['29-07-2023 actualizacion.csv', 'Base.csv']

In [30]:
from datetime import datetime, timedelta
fecha_nombre_anterior = documentos[0].split()[0]
# Convertir a objeto datetime
fecha_nombre_nuevo = datetime.strptime(fecha_nombre_anterior, "%d-%m-%Y") + timedelta(days=1)
# Convertir a objeto str
fecha_nombre_nuevo = fecha_nombre_nuevo.strftime("%d-%m-%Y")
fecha_nombre_nuevo

'30-07-2023'